In [ ]:
import boto3
import os, random, string
from tqdm.notebook import tqdm

In [ ]:
def random_id():
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))

In [ ]:
sqs = boto3.client('sqs')

In [ ]:
queue_url = sqs.get_queue_url(QueueName='SomeRegularQueue')['QueueUrl']
fifo_queue_url = sqs.get_queue_url(QueueName='SomeFifoQueue.fifo')['QueueUrl']

## Create 5000 messages on each queue

In [ ]:
for i in tqdm(range(0, 500)):
    sqs.send_message_batch(QueueUrl=queue_url, Entries=[
        {
            'Id': f'Regular{i}_{j}',
            'MessageBody': f'Hi regular queue {i}/{j}'
        } for j in range(0, 10)
    ])

In [ ]:
# Send messages to the FIFO queue with random groups so we can have more than one in flight at a time

In [ ]:
for i in tqdm(range(0, 500)):
    sqs.send_message_batch(QueueUrl=fifo_queue_url, Entries=[
        {
            'Id': f'Regular{i}_{j}',
            'MessageBody': f'Hi regular queue {i}/{j}',
            'MessageGroupId': random_id(),
            'MessageDeduplicationId': f'{i}_{j}'
        } for j in range(0, 10)
    ])

## Receive Messages
Use a visibility timeout to ensure messages are regarded as 'in-flight', triggering the threshold for
inflight messages percent for the test project.

In [ ]:
sqs.receive_message(QueueUrl=queue_url, VisibilityTimeout=10)

In [ ]:
message_ids_regular = []
sqs.receive_message(QueueUrl=queue_url, VisibilityTimeout=10)
for _ in tqdm(range(0, 1201)):
    msg_id = sqs.receive_message(QueueUrl=queue_url, VisibilityTimeout=1000)['Messages'][0]['MessageId']
    message_ids_regular.append(msg_id)

In [ ]:
sqs.receive_message(QueueUrl=fifo_queue_url, VisibilityTimeout=10)
for _ in tqdm(range(0, 181)):
    resp = sqs.receive_message(QueueUrl=fifo_queue_url, VisibilityTimeout=1000)
    msg_id = resp['Messages'][0]['MessageId']
    message_ids_fifo.append(msg_id)

## Clean up queues

In [ ]:
for url in tqdm([queue_url, fifo_queue_url]):
    sqs.purge_queue(QueueUrl=url)